In [9]:
pip install websockets


[notice] A new release of pip available: 22.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install nest_asyncio


[notice] A new release of pip available: 22.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:
import asyncio
import websockets
import json
import datetime
import os
import nest_asyncio
nest_asyncio.apply()

# 설정값
SYMBOL = "btc_krw"  # 원하는 심볼
DURATION_MINUTES = 1  # 수집 시간 (분)


SAVE_DIR = "orderbook_data"
os.makedirs(SAVE_DIR, exist_ok=True)

async def collect_orderbook():
    url = "wss://ws-api.korbit.co.kr/v2/public"
    end_time = datetime.datetime.now() + datetime.timedelta(minutes=DURATION_MINUTES)
    filename = f"{SAVE_DIR}/orderbook_{SYMBOL}_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.jsonl"

    while datetime.datetime.now() < end_time:
        try:
            async with websockets.connect(url, ping_interval=20) as ws:
                # subscribe 메시지
                subscribe_msg = [{"method":"subscribe","type":"orderbook","symbols":["btc_krw"]}]
                await ws.send(json.dumps(subscribe_msg))
                print(f"✅ WebSocket 연결됨: {SYMBOL}")

                with open(filename, "a") as f:
                    last_saved = datetime.datetime.now()
                    while datetime.datetime.now() < end_time:
                        try:
                            message = await asyncio.wait_for(ws.recv(), timeout=10)
                            #print(f"원본 message: {message}")
                            now = datetime.datetime.now()
                            if (now - last_saved).total_seconds() >= 1.0:
                                data = json.loads(message)
                                timestamp = datetime.datetime.now().isoformat()
                                f.write(json.dumps({"timestamp": timestamp, "data": data}) + "\n")
                                print(f"[{timestamp}] 수신됨")
                                print(f"[{timestamp}] 저장됨")
                                last_saved = now
                            else:
                                pass  # 저장 안 함
                        except asyncio.TimeoutError:
                            print("⏰ 수신 대기 중... 재접속 시도")
                            break  # 밖의 while문으로 나가서 재연결
        except Exception as e:
            print(f"🚨 연결 오류 발생: {e}")
        await asyncio.sleep(1)  # 재연결 딜레이

if __name__ == "__main__":
    loop = asyncio.get_event_loop()
    loop.run_until_complete(collect_orderbook())
    print(f"[{timestamp}] 프로그램 종료")


✅ WebSocket 연결됨: btc_krw
[2025-04-04T13:32:11.227672] 수신됨
[2025-04-04T13:32:11.227672] 저장됨
[2025-04-04T13:32:12.228577] 수신됨
[2025-04-04T13:32:12.228577] 저장됨
[2025-04-04T13:32:13.462846] 수신됨
[2025-04-04T13:32:13.462846] 저장됨
[2025-04-04T13:32:14.553534] 수신됨
[2025-04-04T13:32:14.553534] 저장됨
[2025-04-04T13:32:15.630623] 수신됨
[2025-04-04T13:32:15.630623] 저장됨
[2025-04-04T13:32:16.747087] 수신됨
[2025-04-04T13:32:16.747087] 저장됨
[2025-04-04T13:32:18.029136] 수신됨
[2025-04-04T13:32:18.029136] 저장됨
[2025-04-04T13:32:19.131479] 수신됨
[2025-04-04T13:32:19.131479] 저장됨
[2025-04-04T13:32:20.322079] 수신됨
[2025-04-04T13:32:20.322079] 저장됨
[2025-04-04T13:32:21.330790] 수신됨
[2025-04-04T13:32:21.330790] 저장됨
[2025-04-04T13:32:22.333194] 수신됨
[2025-04-04T13:32:22.333194] 저장됨
[2025-04-04T13:32:23.423014] 수신됨
[2025-04-04T13:32:23.423014] 저장됨
[2025-04-04T13:32:24.431951] 수신됨
[2025-04-04T13:32:24.431951] 저장됨
[2025-04-04T13:32:25.923700] 수신됨
[2025-04-04T13:32:25.923700] 저장됨
[2025-04-04T13:32:27.007152] 수신됨
[2025-04-04T13:32: